In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import torch.nn.functional as F
from imutils import paths

In [2]:
data_transforms = transforms.Compose([
    transforms.Resize([112, 112]),
    transforms.ToTensor()
    ])
torchvision.datasets.GTSRB(root='./data', split="train", download=True, transform=data_transforms)

Dataset GTSRB
    Number of datapoints: 26640
    Root location: ./data
    StandardTransform
Transform: Compose(
               Resize(size=[112, 112], interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )

In [3]:
BATCH_SIZE = 256
learning_rate = 0.001
EPOCHS = 15
numClasses = 43

In [4]:
train_data_path = "./data/gtsrb/GTSRB/Training"
train_data = torchvision.datasets.ImageFolder(root = train_data_path, transform = data_transforms)

ratio = 0.8
n_train_examples = int(len(train_data) * ratio)
n_val_examples = len(train_data) - n_train_examples

train_data, val_data = random_split(train_data, [n_train_examples, n_val_examples])

print(f"Number of training samples = {len(train_data)}")
print(f"Number of validation samples = {len(val_data)}")


trainloader = DataLoader(train_data, shuffle=True, batch_size = BATCH_SIZE)
valloader = DataLoader(val_data, shuffle=True, batch_size = BATCH_SIZE)

Number of training samples = 21312
Number of validation samples = 5328


In [5]:
class SimpleNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=1),
            nn.MaxPool2d(kernel_size=2),

            )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            
            nn.Linear(12544, output_dim)
            )
        
    def forward(self, x):
        x = self.features(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

In [6]:
def evaluate( model, loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for (images, labels) in loader:

            # Run predictions
            output, _ = model(images)
            _, predicted = torch.max(output, 1)
 
            y_pred.extend(predicted.numpy())
            y_true.extend(labels.numpy())

    
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    return accuracy, recall, precision, f1

In [7]:
# Function to perform training of the model

def train(model, loader, opt, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    # Train the model
    model.train()
    
    for (images, labels) in loader:

        
        # Training pass
        opt.zero_grad()
        
        output, _ = model(images)
        loss = criterion(output, labels)
        
        # Backpropagation
        loss.backward()
        

        # Optimizing weights
        opt.step()
        
        epoch_loss += loss.item()

    accuracy, recall, precision, f1 = evaluate(model, loader)

    return epoch_loss / len(loader), accuracy, recall, precision, f1

In [8]:
net = SimpleNet(numClasses)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
val_accuracy = 0

epoch = 1
while val_accuracy < 0.95:
    train_loss, train_accuracy, train_recall, train_precision, train_f1 = train(net, trainloader, optimizer, criterion)

    val_accuracy, val_recall, val_precision, val_f1 = evaluate(net, valloader)

    print(f'Epoch {epoch }, Loss: {train_loss:.3f}')

    print(f'Training - Accuracy: {train_accuracy}, Recall: {train_recall}, Precision: {train_precision}, F1 Score: {train_f1}')

    print(f'Validation - Accuracy: {val_accuracy}, Recall: {val_recall}, Precision: {val_precision}, F1 Score: {val_f1}')
    
    epoch += 1

print('Finished Training')

Epoch 1, Loss: 2.187
Training - Accuracy: 0.6871715465465466, Recall: 0.6299104447135421, Precision: 0.7906369995571911, F1 Score: 0.652625752399974
Validation - Accuracy: 0.6820570570570571, Recall: 0.62505419677975, Precision: 0.7915849368660276, F1 Score: 0.6424018950680908
Epoch 2, Loss: 1.013
Training - Accuracy: 0.8333333333333334, Recall: 0.809662354905736, Precision: 0.8817895275799338, F1 Score: 0.8326978154831096
Validation - Accuracy: 0.81493993993994, Recall: 0.7830919896222606, Precision: 0.8588594831016483, F1 Score: 0.8041306323618749
Epoch 3, Loss: 0.711
Training - Accuracy: 0.8707301051051051, Recall: 0.855414158172392, Precision: 0.9083721768474102, F1 Score: 0.8692778561780958
Validation - Accuracy: 0.8468468468468469, Recall: 0.822368813045981, Precision: 0.8905564626092917, F1 Score: 0.8341833970868942
Epoch 4, Loss: 0.587
Training - Accuracy: 0.8866366366366366, Recall: 0.8919153118467641, Precision: 0.9115054933836596, F1 Score: 0.8959201022155239
Validation - Ac

KeyboardInterrupt: 